Bu yazıda Yolo modellerinin tespit algoritmasının nasıl çalıştığını anlatacağım. Yazıda YoloV3-320 ağırlık ve config dosyalarını kullandım. Başlamadan önce kütüphanelerimizi import edelim.

In [ ]:
import cv2 as cv
import numpy as np

İlk olarak model içerisinde bulunan sınıfların isimlerini barındıran dosyayı içe aktarıp daha sonrasında değerleri bir liste içerisine kayıt edelim. Burada kullanılan coco.names dosyası Yolo'nun kendi modellerini barındıran isim dosyasıdır. Kendi modelleriniz için kendi isim dosyalarınızı oluşturmanız gerekecektir.

In [ ]:
nameOfClasses = []
with open("files/coco.names", "r") as f:
    nameOfClasses = [line.strip() for line in f.readlines()]

print("Yolo Classes",nameOfClasses, end="\n")

Sonrasında model için gerekli olan dosyaları, weights ve config dosyalarını içe aktarıyor ve Darknet üzerinden modelimizi oluşturuyoruz.

In [ ]:
modelConfig = "files/yolov3-320.cfg"
modelWeights = "files/yolov3-320.weights"

model = cv.dnn.readNetFromDarknet(modelConfig, modelWeights)

Modelimizin çalışırken kullanacağı ortam ve cihazları belirtiyoruz. Burada ilk seçenek CPU üzerinden işlem yapmayı sağlar iken ikinci seçenek GPU üzerinden işlem yapmayı sağlar. Ama GPU üzerinden işlem yapabilmek için OpenCV kütüphanesinin CUDA uyumlu şekilde kurulmuş olması gerekmektedir.

In [ ]:
# For CPU
model.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
model.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [ ]:
# For GPU
model.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

Daha sonrasında tespit için kullanacağımız resimi içe aktaralım. İçe aktardığımız resimi model için hazır hale getirmemiz gerekmektedir. Bunu da resmi blob formatına getirerek yaparız. Burada dikkat edilmesi gereken blob resimin boyutudur. Her model belirli bir resim boyutu ile çalışır. Örneğin yolov3-320 320x320, yolov3-tiny 416x416 piksel resimlerile çalışırlar.

In [ ]:
img = cv.imread("01.jpg")
#                                        Resimin Boyutu
blob = cv.dnn.blobFromImage(img, 1 / 255, (320, 320), [0, 0, 0], 1, crop=False)

Artık hazır olan resimimizi model içerisine gönderebiliriz.

In [ ]:
model.setInput(blob)

Bu noktada resimimizi modele gönderdik ve bir sonuç oluştu. Artık yapmamız gereken şey bu oluşan sonuçları modelden çekmek. Bunu yapmamnın yolu ise YoloV3 böral ağında bulunan 3 çıktı katmanını tespit edip tutukları değerleri okumaktır. Bu yüzden ilk olarak nöral ağdaki yani modeldeki tüm katmanları tutan bir değişken oluşturuyoruz.

In [ ]:
layerNames = model.getLayerNames()

print("Katman Sayısı :: ",len(layerNames, "Katman İsimleri :"))
print("\n", layerNames)

Şimdi ise tüm katmanlardan 3 çıktı katmanını ayırma işlemini yapacağız. Bunun için bir for döngüsü içinde tüm katmanları inceliyoruz. Kendinden sonra bir şeye bağlı olmayan yani UnconnectedOutLayer halinde olan katmanlar bizim çıktı katmanlarımızdır ve for döngüsü ile bunlarıda " outputLayerNames " içerisine depoluyoruz. (Burada - List Comperhension - yönetemi kullanılara for döngüsü tek satır haline getirilmiştir.)

In [ ]:
outputLayerNames = [(layerNames[i - 1]) for i in model.getUnconnectedOutLayers()]

print("Çıktı Katmanları ::",outputLayerNames)

Artık çıktı katmanlarını da bildiğimize göre içerlerinde tuttukları sonuçları çekebiliriz.

In [ ]:
outputs =  model.forward(outputLayerNames)

Daha fazla ilerlemeden bilmeyenleriniz için YoloV3'ün ağ yapısından kısaca bahsetmeliyim. YoloV3 nöral ağ yapısı aşağıdaki şekildedir.


Diagramdaki mavi katmanlar bizim kısa süre önce bulduğumuz çıktı katmanlarıdır. Çıktı katmanlarının yapısını incelyeccek olursak, her bir katmanın resim üzerinde belirli sayıda kutusu vardır ve bu kutular içerisinde tanıma işlemi gerçekleştirir. Bu sayede klasik kayan pencereden çok ve çok daha hızlı şekilde tüm resimde tespit denemelerini yapmış olur. Katmanların resim üzerindeki kutularını anlamanız için görselleştirecek olursak eğer


Birazda matematiksel olarak çıktı katmanlarını inceleyelim. 3 çıktı katmanınında boyutlarını inceleyecek olursak eğer . . .

In [ ]:
print(outputs[0].shape)
print(outputs[1].shape)
print(outputs[2].shape)

. . . çıktılarını elde ederiz. Buradaki ilk değerler katmanın kaç kutuya sahip olduğu, ikinci değer ise kaç stünunun olduğudur. Stünlari çıktı katmanlarının buldukları sonuçları depolarlar ama bunu ilerde detaylandıracağız o yüzden kutulara geri dönelim. Eğer tüm çıktı katmanlarındaki kutuları toplarsak tek bir resimde toplam 7300 kutu içerisinde eş zamanlı olrak tespit işlemi gerçekleşiyor. Böylece resimi küçük - büyük, yakın-uzak her açıdan tespite sokmuş oluyoruz. Daha derine inecek olursak eğer . . .

In [ ]:
print(len(outputs))

In [ ]:
print(outputs)

Yukarıdaki listeyi sadeleştirecek olursak eğer bu sonucu elde ederiz :
  1.çıktı katmanını içerisindeki kutuların 85 stündak için değerleri,
 (array([[box1 values],
          [box2 values],
           . . . ]])
  2.çıktı katmanını içerisindeki kutuların 85 stündak için değerleri,
  array([[box1 values],
          [box2 values],
           . . . ]])
  3.çıktı katmanını içerisindeki kutuların 85 stündak için değerleri,
  array([[box1 values],
          [box2 values],
           . . . ]]))

Tek bir katmanın sonuçlarını inceleyecek olursak eğer :

In [1]:
print(outputs[0])

Şimdi ise çıktı katmnanını il kutusunun içerisindeki değerlere bakalım :

In [ ]:
print(outputs[0][0])

Evet işte karşımızda ilk katmanın ilk kutusunun 85 stün için değerleri. Bu 85 stünun neyi temsil ettiğini açıklayayım hemen. İlk 5 stün standarttır ve kutu ile ilgili bilgileri verir. Sırası ile sol üst köşesi için x, y, kutu eni, kutu boyu ve içerisinde bir obje olma olasılığıdır. Geriye kalan 80 stün ise kullandığımız model içerisine yani YoloV3-320 içerinde tespit edilebilir olan objeleri temsil eder. Örneğin 6.stün arabayı temsil eder, tuttuğu değer ise kutu içerisindeki objenin (eğer var ise) arabaya benzerlik yüzdesidir. Bu şekilde geriye kalan 79 sütun da kutu içerisindeki objenin temsil ettikleri objelere benzerlik oranını tutar.

Biz de bu sütunların tuttukları değerlere göre objenin olup olmasığını ve eğer varsa hangi obje olduğunu anlamı oluruz. Tabi biz bu işlemi tüm 7300 kutucuğa yapıyoruz. Bu işlemide ileryeyen kısımlarda koda dökeceğiz tabiki.

Eğer stünlura bir tablo ile temsil edecek olursak eğer :


Burada bilmeniz gereken en öenmli şey ise sütunlardaki en-boy oranlarının 320x320 piksel bir resime göre ayarlanmış olamsıdır yani ilerleyen kısımlarda bu değeri kendi resimimizin boyutuna göre yeniden ayarlayacağız. O zaman başlayalım. İlk olarak resimimizin orjinal en-boy ölçğlerini alalım.

In [ ]:
height, width, channel = img.shape

Sonrasında ise bir obje tespit edilen kutuların bilgilerini saklayacak listeler oluşturalı. Eğer ki istediğimiz oranın üzerisnde bir benzerlik oranı var ise kutunun değerleini bu listelerde saklayacağız.

In [ ]:
boundingBoxes = []
objectsIds = []
confidenceRates = []

Şimdi ise tüm çıktı katmanlarının içerisinden tek tek çıktı katmanını, bu çıktı katmanının da içerisindeki her bir kutuyu inceleme zamanı. Bunun için bir döngü içerisinde döngü açıyoruz.

In [ ]:
for output in outputs:
    for detection in output:

Daha sonrasında ise stünlar içerisindeki en yüksek doğruluk skoruna sahip stünu bulacağız. Bunun için ilk adımımız ilk 5 sütunu işlemden çıkarmak çünkü farklı değerler tutuyorlar. Sonrasında ise np.argmax() ile en yüksek skora sahip stünun indeksini öreniyor ve bunulada sütunun benzerlik oranını çekiyoruz.

In [ ]:
        detectionScores = detection[5:]
        objectId = np.argmax(detectionScores)
        confidence = detectionScores[objectId]

Sonrasında ise eğer bu benzerlik oranı bizim belirlediğimzi oranın üstünde ise, bizim durumumuzda %70, kutunun değerlerini kayıt ediyoruz. Ama öncesinde orjinal resim boyutuna göre yeniden şekillendirerek.

In [ ]:
        if confidence > 0.7:
            w, h = int(detection[2] * width), int(detection[3] * height)
            x, y = int((detection[0] * width) - w / 2), int((detection[1] * height) - h / 2)
            boundingBoxes.append([x, y, w, h])
            objectsIds.append(objectId)
            confidenceRates.append(float(confidence))

Evet artık çıktı katmanlarının sonuçlarından elde ettiğimiz nesne tespiti sonuçlarımız var. Aslında buradan sonra sahip olduğumuz tüm kutuları çizerek işlemi bitirebiliriz ama böyle yaptığımızda yeni bir sorun ile karşılaşacağız buda üst üste binmiş kutular, yani tespit nesnesinin birden fazla kutunun alanı içersinde olmasından dolayı aynı nesneyi tespit etmiş birden fazla kutu. Bunu çözmek içinde OpenCV'nin bize sunduğu NMSBoxes() methodunu kullanacağız. Bu methodun açılımı Non Maximum Supperresion'dur, yaptığı işlem ise üst üsse binen kutuları bularak aralarında benzerlik oranı yüksek oranı saklar, diğerini veya diğerlerini de siler. Böylece temiz bir sonuzcumuz olur.

In [ ]:
indexes = cv.dnn.NMSBoxes(boundingBoxes, confidenceRates, 0.5, 0.2)

Artık elimizdeki temiz sounçlar ile reim üzerinde bu snuçları gösterme zamanı. Bunun için ilk olarak sakladığımız x, y, w ve h değerleri ile bir dikdörtgen çizdiriyoruz sonrasında ise tespit edilen sütunun isim dosyasında ki karşılığını hesaplıyor ve bunuda ekrana yazdırıyoruz. İsim dosyasının önemide tam olarak burada eğer çok sınıflı bir model ile çalışıyor iseniz bu çıkt değerlerinin isimleriş bilmeniz de önemli. Bunun içinde isim dosyalarındaki isimler, sütunlar ile aynı sırada olur ve bu sayede basit bir işlemle daha önce liste içerinde depoladığımız isimlerden sütunun temsil ettiği objenin ismini bulabiliriz.

In [ ]:
for i in indexes:
    box = boundingBoxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    cv.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)
    cv.putText(img, "{}, %{}".format(nameOfClasses[objectsIds[i]].upper(), int(confidenceRates[i] * 100)),
               (x, y - 10), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,255), 2)

Artık geriye kalan tek şey resimi ekran yazdırmak . . .

In [ ]:
cv.imshow('Image', img)
cv.waitKey(0)
cv.destroyAllWindows()